# Applying data minimization to a trained ML model

In this tutorial we will show how to perform data minimization for ML models using the minimization module. 

This will be demonstarted using the Adult dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/adult). 

We use only the numerical features in the dataset because this is what is currently supported by the module.

## Load data

In [4]:
import numpy as np

import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Use only numeric features (age, education-num, capital-gain, capital-loss, hours-per-week)
x_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=(0, 4, 10, 11, 12), delimiter=",")

y_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=14, dtype=str, delimiter=",")


x_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=(0, 4, 10, 11, 12), delimiter=",", skiprows=1)

y_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=14, dtype=str, delimiter=",", skiprows=1)

# Trim trailing period "." from label
y_test = np.array([a[:-1] for a in y_test])

y_train[y_train == ' <=50K'] = 0
y_train[y_train == ' >50K'] = 1
y_train = y_train.astype(int)

y_test[y_test == ' <=50K'] = 0
y_test[y_test == ' >50K'] = 1
y_test = y_test.astype(int)

print(x_train)

[[3.9000e+01 1.3000e+01 2.1740e+03 0.0000e+00 4.0000e+01]
 [5.0000e+01 1.3000e+01 0.0000e+00 0.0000e+00 1.3000e+01]
 [3.8000e+01 9.0000e+00 0.0000e+00 0.0000e+00 4.0000e+01]
 ...
 [5.8000e+01 9.0000e+00 0.0000e+00 0.0000e+00 4.0000e+01]
 [2.2000e+01 9.0000e+00 0.0000e+00 0.0000e+00 2.0000e+01]
 [5.2000e+01 9.0000e+00 1.5024e+04 0.0000e+00 4.0000e+01]]


## Train decision tree model

In [5]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from apt.utils.datasets import ArrayDataset
from apt.utils.models import SklearnClassifier, ModelOutputType
from sklearn.tree import DecisionTreeClassifier

base_est = DecisionTreeClassifier()
model = SklearnClassifier(base_est, ModelOutputType.CLASSIFIER_PROBABILITIES)
model.fit(ArrayDataset(x_train, y_train))

print('Base model accuracy: ', model.score(ArrayDataset(x_test, y_test)))

Base model accuracy:  0.8193599901725939


## Run minimization
We will try to run minimization with different possible values of target accuracy (how close to the original model's accuracy we want to get, 1 being same accuracy as for original data).

In [10]:
from apt.minimization import GeneralizeToRepresentative
from sklearn.model_selection import train_test_split

# default target_accuracy is 0.998
minimizer = GeneralizeToRepresentative(model)

# Feature names for X
feature_names = ['age', 'edu-level', 'capital-gain', 'capital-loss', 'hours-per-week']

# Create a dictionary to map feature names to their corresponding indices
feature_name_map = {name: idx for idx, name in enumerate(feature_names)}

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the 
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training 
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train, x_test, y_generalizer_train, y_test = train_test_split(x_test, y_test, stratify=y_test,
                                                                test_size = 0.4, random_state = 38)


# Set feature names for your X_generalizer_train dataset
X_generalizer_train_with_names = ArrayDataset(X_generalizer_train)

x_train_predictions = model.predict(ArrayDataset(X_generalizer_train))

print(X_generalizer_train)
print(y_generalizer_train)
print(x_test)
print(y_test)
print(X_generalizer_train_with_names.features_names)
print(x_train_predictions)

if x_train_predictions.shape[1] > 1:
    x_train_predictions = np.argmax(x_train_predictions, axis=1)
minimizer.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed = minimizer.transform(dataset=ArrayDataset(x_test))

print('Accuracy on minimized data: ', model.score(ArrayDataset(transformed, y_test)))

[[  32.    9.    0.    0.   40.]
 [  62.   14. 7688.    0.   40.]
 [  24.   13.    0.    0.   40.]
 ...
 [  40.   13.    0.    0.   72.]
 [  31.   11.    0.    0.   40.]
 [  44.   11.    0.    0.   40.]]
[0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0]
[[3.1000e+01 1.3000e+01 0.0000e+00 0.0000e+00 4.0000e+01]
 [3.3000e+01 9.0000e+00 0.0000e+00 0.0000e+00 5.5000e+01]
 [1.7000e+01 7.0000e+00 0.0000e+00 0.0000e+00 1.4000e+01]
 [4.3000e+01 1.0000e+01 0.0000e+00 0.0000e+00 5.5000e+01]
 [4.4000e+01 9.0000e+00 0.0000e+00 0.0000e+00 3.2000e+01

#### Let's see what features were generalized

In [12]:
generalizations = minimizer.generalizations
print(generalizations)

{'ranges': {}, 'categories': {}, 'untouched': ['3', '2', '4', '0', '1'], 'category_representatives': {}, 'range_representatives': {}}


We can see that for the default target accuracy of 0.998 of the original accuracy, no generalizations are possible (all features are left untouched, i.e., not generalized).

Let's change to a slightly lower target accuracy.

In [13]:
# We allow a 1% deviation in accuracy from the original model accuracy
minimizer2 = GeneralizeToRepresentative(model, target_accuracy=0.99)

minimizer2.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed2 = minimizer2.transform(dataset=ArrayDataset(x_test))
print('Accuracy on minimized data: ', model.score(test_data=ArrayDataset(transformed2, y_test)))
generalizations2 = minimizer2.generalizations
print(generalizations2)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.927063
Improving accuracy
feature to remove: 2
Removed feature: 2, new relative accuracy: 0.931542
feature to remove: 0
Removed feature: 0, new relative accuracy: 0.934101
feature to remove: 1
Removed feature: 1, new relative accuracy: 0.952015
feature to remove: 4
Removed feature: 4, new relative accuracy: 0.995521
Accuracy on minimized data:  0.8115886415963162
{'ranges': {'3': [690.0, 782.0, 1526.0, 1588.0, 1665.0, 1779.5, 1824.5, 1978.5]}, 'categories': {}, 'untouched': ['0', '1', '2', '4'], 'category_representatives': {}, 'range_representatives': {'3': [690.0, 0.0, 372.0, 1421.0, 1564.0, 1590.0, 1740.0, 77.0]}}


This time we were able to generalize one feature, feature number 3 (capital-loss).